In [1]:
import pandas as pd
import yfinance as yf

# Load data_raw.csv

In [2]:
data = pd.read_csv('data_raw.csv')

data

,Company,Tahun_Buku,Tanggal Rilis,kas,jumlah_aset_lancar,persediaan,piutang,jumlah_aset_tidak_lancar,jumlah_aset,jumlah_liabilitas_jangka_pendek,...,jumlah_ekuitas,jumlah_liabilitas,pendapatan,cogs,laba_bruto,laba_rugi,laba_rugi_komprehensif,EPS,category,7D_Avg
0,AALI,2020,2021-02-24 14:27:00,9.788920e+05,5.937890e+06,2.165603e+06,1.513218e+06,2.184334e+07,2.778123e+07,1.792506e+06,...,1.924779e+07,8.533437e+06,1.880704e+07,-1.584415e+07,1.462635e+06,8.937790e+05,4.672530e+05,432.84000,General,10057.715402
1,ABBA,2020,2021-08-25 23:48:00,2.672453e+10,6.413759e+10,4.693148e+09,3.208438e+10,1.575117e+11,2.216493e+11,2.352234e+11,...,-1.037107e+11,3.253600e+11,1.586079e+11,-8.206386e+10,-4.815920e+10,-5.833187e+10,-1.498279e+11,-16.30000,General,672.857143
2,ABDA,2020,2021-03-31 16:50:00,6.520837e+08,2.214891e+09,NaN,1.077348e+08,2.628904e+08,2.477782e+09,5.249200e+07,...,1.387236e+09,1.090546e+09,5.728519e+08,-6.733758e+08,1.608748e+08,1.381903e+08,1.707800e+08,223.00000,Insurance,6821.888184
3,ABMM,2020,2021-05-31 15:01:00,1.092124e+08,3.141606e+08,3.771542e+07,1.584556e+08,5.130766e+08,8.272372e+08,2.626915e+08,...,1.617491e+08,6.654880e+08,6.064074e+08,-5.048924e+08,-2.393122e+07,-3.774006e+07,-5.845940e+07,-0.01295,General,675.715995
4,ACES,2020,2021-05-06 17:16:00,2.219785e+12,5.034737e+12,2.453227e+12,3.421895e+11,2.212327e+12,7.247064e+12,8.449281e+11,...,5.222243e+12,2.024821e+12,7.412767e+12,-3.753585e+12,9.233358e+11,7.313106e+11,8.338070e+11,42.86000,General,1329.044887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2240,ZATA,2022,2023-04-03 15:02:00,2.447842e+10,4.568389e+11,NaN,1.166184e+11,2.688539e+11,7.256928e+11,5.706682e+10,...,5.119395e+11,2.137533e+11,1.949582e+11,-1.082183e+11,9.485564e+09,5.768875e+09,5.464071e+09,1.34000,General,67.857143
2241,ZBRA,2022,2023-04-10 17:54:00,1.715849e+10,2.446170e+12,NaN,1.128596e+12,7.101042e+11,3.156274e+12,1.728423e+12,...,1.260000e+12,1.896274e+12,2.958368e+12,-2.607505e+12,-1.097531e+11,-1.123083e+11,-1.092776e+11,46.45000,General,480.285714
2242,ZINC,2022,2023-06-26 21:49:00,5.346490e+10,6.777764e+11,NaN,4.375605e+11,1.798178e+12,2.475954e+12,7.080128e+11,...,7.731216e+11,1.702832e+12,7.173416e+11,-5.399108e+11,-1.288860e+11,-1.147091e+11,-1.141491e+11,-4.08000,General,50.000000
2243,ZONE,2022,2023-04-03 15:09:00,4.910326e+09,3.747793e+11,NaN,4.837608e+10,2.770019e+11,6.517812e+11,1.991748e+11,...,3.468482e+11,3.049330e+11,6.728814e+11,-2.961784e+11,9.707721e+10,7.294051e+10,7.234408e+10,84.00000,General,1196.428571


# Changes Format Number to Float

In [3]:
header_exclude = ['Company','Tahun_Buku','Tanggal Rilis','category','End of Next Month']

header_list_changes =[]

for i in data.columns:
    if i not in header_exclude:
        header_list_changes.append(i)

header_list_changes

data[header_list_changes] = data[header_list_changes].replace({',': ''}, regex=True).astype(float)

In [4]:
def cogs(data):
    data_laba_bruto = data[data['laba_bruto'].isnull()]
    data_bruto = data[~data['laba_bruto'].isnull()]
    data_bruto['cogs'] = data_bruto['laba_bruto'] - data_bruto['pendapatan']
    data_laba_bruto['laba_bruto'] = data_laba_bruto['pendapatan'] + data_laba_bruto['cogs']
    data = pd.concat([data_bruto,data_laba_bruto])
    return data

In [5]:
def piutang_process(data):
    data_piutang = data[(data['piutang'].isnull()) | (data['piutang'] == 0)]
    data_piutang['piutang']=0.000001
    data_piutang_col = data[~((data['piutang'].isnull()) | (data['piutang'] == 0))]
    data = pd.concat([data_piutang,data_piutang_col])
    return data

In [6]:
data = data.drop(columns=['persediaan'])
data = cogs(data)
data['jumlah_aset_tidak_lancar'] = data['jumlah_aset'] - data['jumlah_aset_lancar']
data['jumlah_liabilitas_jangka_panjang'] = data['jumlah_liabilitas'] - data['jumlah_liabilitas_jangka_pendek']
data = data[data['7D_Avg'].notnull()]
data = data[data['pendapatan'].notnull()]
data = piutang_process(data)
#data = data.replace(0, 0.00001)

C:\Users\arsen\AppData\Local\Temp\ipykernel_2864\461462727.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bruto['cogs'] = data_bruto['laba_bruto'] - data_bruto['pendapatan']
C:\Users\arsen\AppData\Local\Temp\ipykernel_2864\461462727.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_laba_bruto['laba_bruto'] = data_laba_bruto['pendapatan'] + data_laba_bruto['cogs']
C:\Users\arsen\AppData\Local\Temp\ipykernel_2864\2518740620.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy 

# Ratio Calculation

In [7]:
# Price Earning Ratio

data['per'] = data['7D_Avg'] / data['EPS']

# Current Ratio
data['current_ratio'] = data['jumlah_aset_lancar'] / data['jumlah_liabilitas_jangka_pendek']

#Quick Ratio
#data['quick_ratio'] = (data['jumlah_aset_lancar'] - data['persediaan']) / data['jumlah_liabilitas_jangka_pendek']

#Cash Ratio 
data['cash_ratio'] = data['kas'] / data['jumlah_liabilitas_jangka_pendek']

#Account Receivable Turnover
data['AR_Turnover'] = data['pendapatan']/data['piutang']

#Account Receivable Turnover in Days
data['AR_Turnover_in_Days'] = int(365) / data['AR_Turnover']

#Asset Turnover
data['Aset_Turnover'] = data['pendapatan'] / data['jumlah_aset']

#Inventory Turnover
#data['inventory_turnover'] = data['cogs'] / data['persediaan']

#deb to Equity
data['debt_to_equity'] = data['jumlah_liabilitas'] / data['jumlah_ekuitas']

#equity ratio
data['equity_ratio'] = data['jumlah_ekuitas'] / data['jumlah_aset']

#Debt Ratio --> Is it needed? even already had equity ratio?
data['debt_ratio'] = data['jumlah_liabilitas'] / data['jumlah_aset']

#Gross Profit Margin
data['gpm'] = data['laba_bruto'] / data['pendapatan']

#Net Profit Margin
data['net_profit_margin'] = data['laba_rugi_komprehensif'] / data['pendapatan']

#Return on Asset
data['roa'] = data['laba_rugi_komprehensif'] / data['jumlah_aset']

#return on Capital Employed
data['roce'] = data['laba_bruto']/(data['jumlah_ekuitas']+data['jumlah_liabilitas_jangka_panjang'])


In [8]:
data.to_csv('data_olahan.csv', index=False)

# Number of nan

In [9]:
number_of_missing_value = data.isna().sum()

In [10]:
number_of_missing_value



Company                             0
Tahun_Buku                          0
Tanggal Rilis                       0
kas                                 0
jumlah_aset_lancar                  0
piutang                             0
jumlah_aset_tidak_lancar            0
jumlah_aset                         0
jumlah_liabilitas_jangka_pendek     0
jumlah_liabilitas_jangka_panjang    0
jumlah_ekuitas                      0
jumlah_liabilitas                   0
pendapatan                          0
cogs                                0
laba_bruto                          0
laba_rugi                           0
laba_rugi_komprehensif              0
EPS                                 0
category                            0
7D_Avg                              0
per                                 0
current_ratio                       0
cash_ratio                          0
AR_Turnover                         0
AR_Turnover_in_Days                 0
Aset_Turnover                       0
debt_to_equi

In [11]:
len(data)

2233